In [1]:
import os
import re
import sys
import torch
import pprint
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.interpolate import make_interp_spline

__file__ = os.path.dirname(os.path.realpath("__file__"))
root_dir = (Path(__file__).parent / "..").resolve()
lib_dir = (root_dir / "lib").resolve()
print("The root path: {:}".format(root_dir))
print("The library path: {:}".format(lib_dir))
assert lib_dir.exists(), "{:} does not exist".format(lib_dir)
if str(lib_dir) not in sys.path:
    sys.path.insert(0, str(lib_dir))
from utils.qlib_utils import QResult

The root path: /Users/xuanyidong/Desktop/AutoDL-Projects
The library path: /Users/xuanyidong/Desktop/AutoDL-Projects/lib


In [2]:
qresults = torch.load(os.path.join(root_dir, 'notebooks', 'TOT', 'temp-time-x.pth'))
for qresult in qresults:
    print(qresult.name)
all_dates = set()
for qresult in qresults:
    dates = qresult.find_all_dates()
    for date in dates:
        all_dates.add(date)
all_dates = sorted(list(all_dates))
print('There are {:} dates'.format(len(all_dates)))
print('Dates: {:} {:}'.format(all_dates[0], all_dates[1]))

TSF-2x24-drop0_0s2013-01-01
TSF-2x24-drop0_0s2012-01-01
TSF-2x24-drop0_0s2008-01-01
TSF-2x24-drop0_0s2009-01-01
TSF-2x24-drop0_0s2010-01-01
TSF-2x24-drop0_0s2011-01-01
TSF-2x24-drop0_0s2008-07-01
TSF-2x24-drop0_0s2009-07-01
There are 3011 dates
Dates: 2008-01-02 2008-01-03


In [3]:
import matplotlib
from matplotlib import cm
matplotlib.use("agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [6]:
def vis_time_curve(qresults, dates, use_original, save_path):
    save_dir = (save_path / '..').resolve()
    save_dir.mkdir(parents=True, exist_ok=True)
    print('There are {:} qlib-results'.format(len(qresults)))
    
    dpi, width, height = 200, 5000, 2000
    figsize = width / float(dpi), height / float(dpi)
    LabelSize, LegendFontsize = 22, 12
    font_gap = 5
    linestyles = ['-', '--']
    colors = ['k', 'r']
    
    fig = plt.figure(figsize=figsize)
    cur_ax = fig.add_subplot(1, 1, 1)
    for idx, qresult in enumerate(qresults):
        print('Visualize [{:}] -- {:}'.format(idx, qresult.name))
        x_axis, y_axis = [], []
        for idate, date in enumerate(dates):
            if date in qresult._date2ICs[-1]:
                mean, std = qresult.get_IC_by_date(date, 100)
                if not np.isnan(mean):
                    x_axis.append(idate)
                    y_axis.append(mean)
        x_axis, y_axis = np.array(x_axis), np.array(y_axis)
        if use_original:
            cur_ax.plot(x_axis, y_axis, linewidth=1, color=colors[idx], linestyle=linestyles[idx])
        else:
            xnew = np.linspace(x_axis.min(), x_axis.max(), 200)
            spl = make_interp_spline(x_axis, y_axis, k=5)
            ynew = spl(xnew)
            cur_ax.plot(xnew, ynew, linewidth=2, color=colors[idx], linestyle=linestyles[idx])
        
    for tick in cur_ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
    for tick in cur_ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
    cur_ax.set_ylabel("IC (%)", fontsize=LabelSize)
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight", format="pdf")
    plt.close("all")

In [7]:
# Visualization
home_dir = Path.home()
desktop_dir = home_dir / 'Desktop'
print('The Desktop is at: {:}'.format(desktop_dir))

vis_time_curve(
    (qresults[2], qresults[-1]),
    all_dates,
    True,
    desktop_dir / 'es_csi300_time_curve.pdf')

vis_time_curve(
    (qresults[2], qresults[-1]),
    all_dates,
    False,
    desktop_dir / 'es_csi300_time_curve-inter.pdf')

The Desktop is at: /Users/xuanyidong/Desktop
There are 2 qlib-results
Visualize [0] -- TSF-2x24-drop0_0s2008-01-01
Visualize [1] -- TSF-2x24-drop0_0s2009-07-01
There are 2 qlib-results
Visualize [0] -- TSF-2x24-drop0_0s2008-01-01
Visualize [1] -- TSF-2x24-drop0_0s2009-07-01
